## Парсер железнодорожных станций, основываясь на необходимых параметрах

### Импорт библиотек

In [26]:
import pandas as pd
import numpy as np

### Чтение файлов, содержащих информацию о железнодорожных станциях

Источник данных файлов: http://osm.sbin.ru/esr/   
© Идея принадлежит [Sergey Gladilin](https://www.openstreetmap.org/user/Sergey%20Gladilin). Разработка и реализация - [Sergey Gladilin](https://www.openstreetmap.org/user/Sergey%20Gladilin) и [Alexandr Zeinalov](http://www.openstreetmap.org/user/Alexandr%20Zeinalov).   
Использовано по лицензии GNU GPLv3

In [27]:
esr_df = pd.read_csv('esr.csv', sep=';')
osm2esr_df = pd.read_csv('osm2esr.csv', sep=';')

### Предпросмотр импортированных файлов

In [28]:
esr_df

,division,esr,invalid_esr,country,region,express,dup_esr,source,iso3166,railway,type,name
0,NaN,15,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Московский узел
1,NaN,23,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Санкт-Петербургский узел
2,NaN,5201,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Кавказ-Варна (условный код)
3,NaN,5322,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Котка
4,NaN,7627,1,NaN,Латвия,NaN,97627.0,latvia,LV,Латвийская,NaN,Оп Апшупе
...,...,...,...,...,...,...,...,...,...,...,...,...
26639,NaN,997911,0,NaN,Сахалинская область,2068462.0,NaN,sakhalin,RU-SAK,Сахалинская,остановочный пункт,572 км
26640,NaN,997926,0,NaN,Сахалинская область,2068402.0,NaN,sakhalin,RU-SAK,Сахалинская,остановочный пункт,Арги-Паги
26641,NaN,998007,0,Российская Федерация,Сахалинская область,2068459.0,NaN,sakhalin,RU-SAK,Сахалинская,станция,Ныш
26642,NaN,998100,0,Российская Федерация,Сахалинская область,2068498.0,NaN,sakhalin,RU-SAK,Сахалинская,станция,Ноглики


In [29]:
osm2esr_df

,esr,status,type,osm_id,lat,lon,name,alt_name,old_name,official_name,railway,user
0,10002,1,0,665115765,61.784218,34.344021,Петрозаводск-Пассажирский,NaN,NaN,NaN,station,010002
1,10318,1,0,370804,61.886433,34.249794,Шуйский Мост,NaN,NaN,NaN,halt,010318
2,10407,1,0,370656,61.941113,34.250393,Шуйская,NaN,NaN,NaN,station,010407
3,10411,1,0,1328926694,61.980099,34.245548,427 км,NaN,NaN,NaN,halt,010411
4,10426,1,0,1424435426,62.037678,34.214035,Лучевой,NaN,NaN,NaN,station,010426
...,...,...,...,...,...,...,...,...,...,...,...,...
19952,997907,1,0,1825940876,51.318710,142.714294,Альба,NaN,NaN,NaN,station,NaN
19953,997926,1,0,1024315848,51.339256,142.719803,Арги-Паги,NaN,NaN,NaN,halt,NaN
19954,998007,1,0,1815735211,51.495022,142.790359,Ныш,NaN,NaN,NaN,station,NaN
19955,998100,1,0,1825940877,51.797131,143.129868,Ноглики,NaN,NaN,NaN,station,998100


### Объединение двух DataFrame на основании общей столбца `esr` - индивидуального номера каждой станции

In [30]:
stations_df = pd.merge(esr_df, osm2esr_df, on='esr', how='outer')
# удаляем столбцы, не несущие пользы в результирующей таблице
stations_df = stations_df.drop(labels=['invalid_esr', 'alt_name', 'old_name', 'official_name', 'user', 'dup_esr', 'status'], axis=1)
# если нужны только станции с наличием GPS координат - раскомментируйте строку ниже
# stations_df = stations_df.dropna(subset=['lat', 'lon'])

У нас образовалось довольно много дублирующих столбцов. ИХ необходимо объединить и удалить ненужные

In [31]:
list(stations_df)

['division',
 'esr',
 'country',
 'region',
 'express',
 'source',
 'iso3166',
 'railway_x',
 'type_x',
 'name_x',
 'type_y',
 'osm_id',
 'lat',
 'lon',
 'name_y',
 'railway_y']

### Работа с дублирующимися столбцами

In [32]:
# дозаполняем пустые ячейки в двух столбцах "параллельно"
stations_df['name_x'] = stations_df['name_x'].fillna(stations_df['name_y'])
stations_df['name_y'] = stations_df['name_y'].fillna(stations_df['name_x'])
# оставляем только один из столбцов
stations_df['name'] = stations_df['name_y']
stations_df = stations_df.drop(labels=['name_y', 'name_x'], axis=1)
stations_df
# stations_df[['name_x', 'name_y']]

,division,esr,country,region,express,source,iso3166,railway_x,type_x,type_y,osm_id,lat,lon,railway_y,name
0,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Московский узел
1,NaN,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Санкт-Петербургский узел
2,NaN,5201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Кавказ-Варна (условный код)
3,NaN,5322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Котка
4,NaN,7627,NaN,Латвия,NaN,latvia,LV,Латвийская,NaN,NaN,NaN,NaN,NaN,NaN,Оп Апшупе
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,NaN,997926,NaN,Сахалинская область,2068402.0,sakhalin,RU-SAK,Сахалинская,остановочный пункт,0.0,1.024316e+09,51.339256,142.719803,halt,Арги-Паги
28328,NaN,998007,Российская Федерация,Сахалинская область,2068459.0,sakhalin,RU-SAK,Сахалинская,станция,0.0,1.815735e+09,51.495022,142.790359,station,Ныш
28329,NaN,998100,Российская Федерация,Сахалинская область,2068498.0,sakhalin,RU-SAK,Сахалинская,станция,0.0,1.825941e+09,51.797131,143.129868,station,Ноглики
28330,NaN,998100,Российская Федерация,Сахалинская область,2068498.0,sakhalin,RU-SAK,Сахалинская,станция,1.0,2.119020e+08,51.797001,143.130157,station,Ноглики


In [33]:
# по аналогии с предыдущей ячейкой продолжаем заполнение столбцов
stations_df[['railway_x', 'railway_y']]
stations_df['station_type_en'] = stations_df['railway_y']
stations_df['railway'] = stations_df['railway_x']
stations_df = stations_df.drop(labels=['railway_x', 'railway_y'], axis=1)
stations_df

,division,esr,country,region,express,source,iso3166,type_x,type_y,osm_id,lat,lon,name,station_type_en,railway
0,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Московский узел,NaN,NaN
1,NaN,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Санкт-Петербургский узел,NaN,NaN
2,NaN,5201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Кавказ-Варна (условный код),NaN,NaN
3,NaN,5322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Котка,NaN,NaN
4,NaN,7627,NaN,Латвия,NaN,latvia,LV,NaN,NaN,NaN,NaN,NaN,Оп Апшупе,NaN,Латвийская
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,NaN,997926,NaN,Сахалинская область,2068402.0,sakhalin,RU-SAK,остановочный пункт,0.0,1.024316e+09,51.339256,142.719803,Арги-Паги,halt,Сахалинская
28328,NaN,998007,Российская Федерация,Сахалинская область,2068459.0,sakhalin,RU-SAK,станция,0.0,1.815735e+09,51.495022,142.790359,Ныш,station,Сахалинская
28329,NaN,998100,Российская Федерация,Сахалинская область,2068498.0,sakhalin,RU-SAK,станция,0.0,1.825941e+09,51.797131,143.129868,Ноглики,station,Сахалинская
28330,NaN,998100,Российская Федерация,Сахалинская область,2068498.0,sakhalin,RU-SAK,станция,1.0,2.119020e+08,51.797001,143.130157,Ноглики,station,Сахалинская


In [34]:
stations_df[['type_x', 'type_y']]
# stations_df['station_type_ru'] = stations_df['type_x']
stations_df = stations_df.drop(labels=['type_x', 'type_y'], axis=1)
stations_df

,division,esr,country,region,express,source,iso3166,osm_id,lat,lon,name,station_type_en,railway
0,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Московский узел,NaN,NaN
1,NaN,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Санкт-Петербургский узел,NaN,NaN
2,NaN,5201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Кавказ-Варна (условный код),NaN,NaN
3,NaN,5322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Котка,NaN,NaN
4,NaN,7627,NaN,Латвия,NaN,latvia,LV,NaN,NaN,NaN,Оп Апшупе,NaN,Латвийская
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28327,NaN,997926,NaN,Сахалинская область,2068402.0,sakhalin,RU-SAK,1.024316e+09,51.339256,142.719803,Арги-Паги,halt,Сахалинская
28328,NaN,998007,Российская Федерация,Сахалинская область,2068459.0,sakhalin,RU-SAK,1.815735e+09,51.495022,142.790359,Ныш,station,Сахалинская
28329,NaN,998100,Российская Федерация,Сахалинская область,2068498.0,sakhalin,RU-SAK,1.825941e+09,51.797131,143.129868,Ноглики,station,Сахалинская
28330,NaN,998100,Российская Федерация,Сахалинская область,2068498.0,sakhalin,RU-SAK,2.119020e+08,51.797001,143.130157,Ноглики,station,Сахалинская


### Проводим выборку только железнодорожных станций, находящихся на территории РФ

In [35]:
# stations_df['country'] = stations_df['country'].fillna(value='NAN')
# stations_df[(stations_df['country'] == 'Российская Федерация') | (stations_df['country'] == 'NAN')]
stations_df = stations_df[(stations_df['country'] == 'Российская Федерация') | (stations_df['iso3166'].str.contains('RU'))]
stations_df = stations_df.drop(labels=['country'], axis=1)
stations_df

,division,esr,region,express,source,iso3166,osm_id,lat,lon,name,station_type_en,railway
5,Петрозаводское,10002,Республика Карелия,2004300.0,karel,RU-KR,6.651158e+08,61.784218,34.344021,Петрозаводск-Пассажирский,station,Октябрьская
8,NaN,10110,Республика Карелия,NaN,karel,RU-KR,NaN,NaN,NaN,ПЕТРОЗАВОДСК-ГОЛ,NaN,Октябрьская
9,NaN,10125,Республика Карелия,NaN,karel,RU-KR,NaN,NaN,NaN,ПЕТРОЗАВОДСК-ТОМ,NaN,Октябрьская
13,Петрозаводское,10205,Республика Карелия,2004775.0,karel,RU-KR,NaN,NaN,NaN,ГОЛИКОВКА,NaN,Октябрьская
14,Петрозаводское,10214,Республика Карелия,2004775.0,karel,RU-KR,NaN,NaN,NaN,Голиковка,NaN,Октябрьская
...,...,...,...,...,...,...,...,...,...,...,...,...
28327,NaN,997926,Сахалинская область,2068402.0,sakhalin,RU-SAK,1.024316e+09,51.339256,142.719803,Арги-Паги,halt,Сахалинская
28328,NaN,998007,Сахалинская область,2068459.0,sakhalin,RU-SAK,1.815735e+09,51.495022,142.790359,Ныш,station,Сахалинская
28329,NaN,998100,Сахалинская область,2068498.0,sakhalin,RU-SAK,1.825941e+09,51.797131,143.129868,Ноглики,station,Сахалинская
28330,NaN,998100,Сахалинская область,2068498.0,sakhalin,RU-SAK,2.119020e+08,51.797001,143.130157,Ноглики,station,Сахалинская


### Экспортируем готовую таблицу в csv

In [36]:
stations_df.to_csv('RU_stations_new.csv', sep='\t', encoding='utf-8')